## Introduction to Low-Level Image Processing

You can get a long way in image processing and computer vision with
fairly simple operations that basically just work with arithmetic.
They're not only easier to use than more complex algorithms, but 
also usually faster, easier to inspect, and more reliable than 
complex algorithms. 

It's good to understand low-level techniques if you think you might want 
to use more complex techniques for image recognition or processing, 
including machine learning methods, because complex techniques often 
don't work well without preprocessing by simpler functions. Simple 
functions can also be assembled into quite complex ones!

In those notebook, we'll primarily be using "morphological" operations 
-- operations that work with line, shape, and form. However, we'll start
with a little introduction to their extended family.

### Footprint Operations

Many image manipulation techniques rely on looking at each pixel in the 
image, then applying some kind of mathematical function to the pixels 
that fall within some region around it. This region is called a "footprint"
(or sometimes a "kernel").  

Let's look briefly at a useful _non_-morphological example of this
technique: the median filter. This filter works by replacing each pixel 
in an image with the median value of the pixels that fall within a footprint
around it.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# NOTE: I don't remember if we opened 'desktop' images before.
from PIL import Image
import scipy.ndimage as ndi
# This is a little settings file that removes all the annoying
# borders and axes and things that aren't really relevant
# when we're just using matplotlib to look at things as images.
plt.style.use('settings/simple_image.mplstyle')

In [ ]:
# load in a noisy cross shape.
noisecross = np.asarray(Image.open('images/noisecross.png'))
_ = plt.imshow(noisecross)

In [ ]:
# generate big and small footprints.
# note that what's 'big' and 'small' depends on the
# size of the image you're applying a filter to!
fbig, fsmall = 10, 3
big_foot, small_foot = np.ones((fbig, fbig)), np.ones((fsmall, fsmall))
# these are just arrays:
small_foot

In [ ]:
# The median filter has a 'softening' effect. When the footprint is small, the 
# softening is relatively mild; as it gets bigger, the softening gets more intense.
# This is a little bit like turning up the bass on a piece of music: it emphasizes
# more consistent parts of the image and suppresses more variable ones.
fig, axes = plt.subplots(1, 3, figsize=(10, 4))
axes[0].imshow(noisecross)
axes[1].imshow(ndi.median_filter(noisecross, footprint=small_foot))
axes[2].imshow(ndi.median_filter(noisecross, footprint=big_foot))
for i, title in zip((0, 1, 2), ("original", "median_small", "median_big")):
    axes[i].set_title(title)

In [ ]:
# This can be used for many practical purposes. For instance, look 
# at what a median filter can do to the scanlines in this classic Viking Orbiter image
# (a technique the ground team used to great advantage!):
import pdr
viking = pdr.read("/datascratch/viking/edr/vo_1023/f611axx/F611A13.IMG").IMAGE
fig, axes = plt.subplots(1, 2, figsize=(15, 9))
axes[0].imshow(viking)
axes[1].imshow(ndi.median_filter(viking, footprint=np.ones((4, 4))))

## Morphological Operators

Because subjective fullness of the Moon is basically about shape and line,
we'd like to be able to work directly with those aspects of images. 
Morphological operators are powerful tools for doing this. Morphological
operators are a type of footprint-based filter that use logical operations
like "and" and "or" instead of arithmetic.

### Making Binary Images

Because they do true/false logic, morphological operators want to work on
"binary" images -- black-and-white images made up of only 1s and 0s.

Most images we want to work with don't start out as binary images. The
easiest (and one of the most effective) ways to reduce images to lines
is to set a cutoff value or "threshold". We then set all values below
that threshold to black, and all values above that threshold to white. 
If it's a color image, we also want to turn it to grayscale first.
This often works something like tracing or making an outline of an image --
and these are also good first steps in other processes that want outlines,
like silkscreening.

Let's go ahead and walk through the process.

In [ ]:
# Here is a detail of part of a Tiffany lamp.
tiffany = np.asarray(Image.open("images/tiffany.png"))
plt.imshow(tiffany)

In [ ]:
# Color images are usually 3-D arrays where the third axis
# represents color channel, in this case red (R), green (G) or blue (B).
# This would make a purple version of the image:
purple_tiffany = tiffany.copy()
purple_tiffany[:, :, 1] = 0
plt.imshow(purple_tiffany)

In [ ]:
from marslab.imgops.imgutils import eightbit, enhance_color

In [ ]:
plt.imshow(enhance_color(tiffany, (0, 1), 1))

In [ ]:
# The easiest way to make a gray version of a color image is 
# just to merge its channels down, which can be as simple as
# taking their mean or median:
tiffany_gray = np.mean(tiffany, axis=2)
plt.imshow(tiffany_gray)

In [ ]:
# We can now turn it to a black-and-white image with thresholding.
# Picking the correct threshold value is a little bit of an art and
# depends on exactly what features you want the outline to
# retain. Let's see what happens at a few different levels...
fig, axes = plt.subplots(1, 4, figsize=(13, 8))
threshold_levels = (10, 25, 50, 160)
for i, level in enumerate(threshold_levels):
    axes[i].imshow(tiffany_gray > level)
    axes[i].set_title(level)

In [ ]:
# Let's try the second one for our outline -- but 
# you might to try the next steps a few times with different
# settings to see what happens.
tiffany_outline = tiffany_gray > 25

### Dilation and Erosion

There are two basic morphological operators: erosion and dilation. Most
other can be assembled from combinations of these two.

Dilation is an "or". If there is a pixel valued 1 anywhere in the dilation 
operator's footprint, it sets the center pixel to 1; otherwise, it sets it
to 0. Erosion is an "and": if _all_ pixels in the erosion operator's 
footprint are 1, it sets the center pixel to 1; otherwise, it sets the
center pixel to 0.

This means that erosion will tend to make black parts of the image heavier,
thicker, and more coherent, and dilation will do the opposite.

Let's look at erosion and dilation here. Note that, just like the median filter 
we saw earlier, bigger footprints tend to make morphological operatorsand selecting the correct footprint size
or shape for particular images and applications can be a very finicky job. "stronger". 

In [ ]:
# Because this particular image is basically a black-on-white outline, erosion 
# will tend to make its lines thicker and its sections more distinct:
small, big = 4, 9
fig, axes = plt.subplots(1, 2, figsize=(6, 8))
for i, size in enumerate((small, big)):
    footprint = np.ones((size, size))
    axes[i].imshow(ndi.binary_erosion(tiffany_outline, footprint))

In [ ]:
# Whereas dilation will tend to make its lines thinner and blur sections:
fig, axes = plt.subplots(1, 2, figsize=(6, 8))
for i, size in enumerate((small, big)):
    footprint = np.ones((size, size))
    axes[i].imshow(ndi.binary_dilation(tiffany_outline, footprint))

### Labeling

Being able to manipulate lines like this is very useful in part because it
lets us automatically segment images. The simplest way to do this is to
simply find contiguous regions of the image and assign a unique number to 
them. This process is called "labeling", and many good tools for it exist
in Python.

In the next cell, you'll 'thicken' the Tiffany outline and then use `ndi.label()` 
to assign every contiguous 1-valued region of that outline its own unique number.
It always assigns 0 to what it identifies as "background". 
You'll see that it doesn't work perfectly -- it may cut some sections at the 
edges off, and can't quite distinguish some regions that might look contiguous to 
your eye because of some junky little line bits that connect them.

Like threshold levels, selecting good footprint sizes and shapes for particular images
and applications can be a very finicky job. If you change the footprint size
from (3, 3) and run the cell again, you will see that you get quite different labels.
If you make it a lot bigger, you'll see that you get _fewer_ labels, because the erosion
will have made image areas more distinct from one another...although if you make it big 
enough, you might not get any at all, because you'll have eroded all the lines into one
big line. You'll also get very different results -- probably _much_ larger and more 
connected labels -- if you swap it to binary dilation.

In [ ]:
trace = ndi.binary_erosion(tiffany_outline, np.ones((3, 3)))
labels, n_labels = ndi.label(trace)
print(f"{n_labels} labels found.")
plt.imshow(np.ma.masked_where(labels == 0, labels), cmap='flag', interpolation='none')

We can use this to pick out, count, and locate individual
regions of an image. Try running this next cell a few times to drill
down into the specific image elements our little algorithm identified.
You'll note that not every label is interesting -- `ndi.label()`
will happily assign a unique label to a tiny little dot, as long
as it's isolated. When we look at lunar images a little later, 
we'll need to watch for that fact.

In [ ]:
# Utility functions for visualization.

def apply_2d_stencil(color_image, stencil_2d):
    # reshape `stencil_2d` to fit the 3D color image
    stencil_3d = np.moveaxis(np.tile(stencil_2d, (3, 1, 1)), 0, 2)
    # return a 3D array that contains the original pixels where 
    # `stencil_2d` is truthy, and is black elsewhere
    return np.where(stencil_3d, color_image, 0)

def labelstencil(color_image, label_array, label_numbers):
    """
    Return a copy of `color_image` blacked out wherever the values of 
    `label_array` don't fall within `label_numbers`.
    """
    # make an array that's True for these labels and false otherwise
    return apply_2d_stencil(color_image, np.isin(label_array, label_numbers))


# select 8 random numbers from among all labels, not counting the 
# 0/background label.
random_label_choices = np.random.choice(range(1, n_labels), 8)
fig, ax = plt.subplots(figsize=(5, 7))
ax.imshow(labelstencil(tiffany, labels, random_label_choices))

In [ ]:
# you can also look at just the background:
fig, ax = plt.subplots(figsize=(5, 7))
ax.imshow(labelstencil(tiffany, labels, [0]))

### Getting shapes from labels

Because of image defects and the messy imperfection of the world,
labels don't usually _exactly_ fit the regions you'd like to find
in images, and they also often have very complex boundaries that 
make some kinds of analysis harder (even when the actual objects 
in the scene don't). One straightforward technique that's effective
in many situations is simply to draw some simple geometric shape 
like a circle or rectangle around the label. (Sometimes you're not
looking for simple geometric shapes, of course, in which case this
would not be a good idea.)


OpenCV has several fast, reliable functions for doing this.
Its Python wrapper, `cv2`, is very effective, but its expectations 
are unusual and its error messages aren't always very readable (it's 
practically like a separate language). Rather than spend the rest of 
this Notebookjust talking about the OpenCV API, we've simply provided
a couple of example functions in the next cell for performing common 
tasks of this kind with `cv2`.

In [ ]:
import cv2

def mask2vec(arr):
    """
    Converts the truthy elements of an ndarray into an array of "vectors"
    suitable for use by OpenCV geometry functions.
    """
    return np.vstack(np.nonzero(arr.T)).T

def mask2shape(
    mask, 
    shape = "circle", 
    color = (0, 255, 255), 
    thickness = 2,
    draw_on_array = False
):
    """
    Draws the smallest possible circle, triangle, or rectangle around the 
    truthy elements of an ndarray. Returns both the shape parameters and the 
    shape drawn in an ndarray. If draw_on_mask is True, draws the shape
    on top of the existing elements of the mask.
    """
    if shape not in {"circle", "triangle", "rectangle"}:
        raise ValueError("Shape can be 'circle', 'triangle', or 'rectangle'.")
    vec, param = mask2vec(mask), {}
    if draw_on_array is False:
        canvas = np.zeros([*mask.shape, 3], 'u1')
    else:
        canvas = np.moveaxis(
            np.tile(np.where(mask, 255, 0).astype('u1'), [3, 1, 1]), 2, 1
        ).T.copy()
    if shape == "circle":
        (param['cx'], param['cy']), param['r'] = cv2.minEnclosingCircle(vec)
        cx, cy, r = map(lambda p: int(round(p)), param.values())
        return param, cv2.circle(canvas, (cx, cy), r, color, thickness)
    if shape == "triangle":
        param['area'], param['points'] = cv2.minEnclosingTriangle(vec)
        return param, cv2.drawContours(
            canvas, [param['points'].round().astype('i4')], 0, color, thickness
        )
    param['ul'], param['lr'], param['angle'] = cv2.minAreaRect(vec)
    return param, cv2.drawContours(
        canvas, 
        [cv2.boxPoints(tuple(param.values())).round().astype('i4')],
        0,
        color,
        thickness
    )

In [ ]:
# and a simple example:
random_label_choices = np.random.choice(range(1, n_labels), 8)
assembled = np.zeros_like(tiffany)
for label in random_label_choices:
    _, drawn = mask2shape(labels == label, "triangle", thickness=7)
    assembled += drawn
fig, ax = plt.subplots(figsize=(5, 6))
ax.imshow(
    np.clip(assembled + labelstencil(tiffany, labels, random_label_choices), 0, 255)
)

# SECTIONS SECTIONS SECTIONS

# Moon fullness identification algorithm

## Assumptions

These are not completely true, but good enough for the algorithm:
1. A full Moon looks like a perfect circle.
2. The only things that ever appear in the GOES LUN images are stars, the Moon, the Earth, and static.
3. The Earth will always appear as a contiguous region that touches at least one image edge.
4. If the Moon touches an image edge, we can ignore it, because we won't be able to tell how full
   it is anyway.
5. The images are taken at about the same distance from the Moon, so the Moon will fall within a
   small size range.
6. There are no legitimate images of the Moon at less than half full in this set, so we can
   always assume the Moon label is convex.

## Steps

Given those assumptions, we can perform the following steps to figure out how full
the Moon looks in a GOES image:

1. Turn the image into an outline using thresholding.
2. Apply an erosion operator to the outline to clean up static, stars, and imaging imperfections.
3. Label the eroded outline.
4. Exclude labels that are:
    1. the wrong size
    2. too close to an image edge
5. If we have exactly one label left, it's the Moon; proceed to step 6. Otherwise, there's no Moon in the image.
6. Apply a dilation operator to smooth the edges of the Moon label, and then 
7. Draw a circle around the label. The ratio of the area of that circle to the that ratio is
   fullness.

## Procedure

First, we'll build the algorithm step by step together -- but this time, we'll be asking _you_ to 
w e  appropriate parameters for the functions. Hints are available, but there's no strict
right answer to this -- different sets of parameters might work equally well, and some might
work better on some images and worse on others.

Then, we'll try the algorithm on all the images and test it against "ground truth". If you don't
like the results you get, then you can go back and tweak your parameters until you do!

In [ ]:
import cv2
import numpy as np
import scipy.ndimage as ndi
from typing import Union

def a2c(area: Union[float, np.ndarray]) -> Union[float, np.ndarray]:
    """Area of a circle."""
    return 2 * np.pi * np.sqrt(area / np.pi)



def filter_labels(
    arr: np.ndarray, 
    labels: np.ndarray, 
    bordersize: int, 
    min_moon_extent: int
) -> tuple[dict, dict, dict]:
    """
    Filter 'bad' labels: labels that are too small or that come 
    within a specified range of the top/bottom/left/right of the image.
    """
    label_indices = {
        label: np.nonzero(labels == label)
        for label in np.unique(labels)
        if label != 0
    }
    label_rejects = {}
    for label, (yix, xix) in label_indices.items():
        if min([np.ptp(xix), np.ptp(yix)]) < min_moon_extent:
            label_rejects[label] = 'small'
        elif (xix <= bordersize).any():
            label_rejects[label] = 'left'
        elif (arr.shape[1] - xix <= bordersize).any():
            label_rejects[label] = 'right'
        elif (yix <= bordersize).any():
            label_rejects[label] = 'top'
        elif (arr.shape[0] - yix <= bordersize).any():
            label_rejects[label] = 'bottom'
    remaining = {
        k: v for k, v in label_indices.items()
        if k not in label_rejects.keys()
    }
    return remaining, label_rejects, label_indices


def moon_extraction_error(rec: dict, n_remaining: int) -> dict:
    if n_remaining == 0:
        return rec | {'error': 'no Moon', 'moon': None}
    if n_remaining > 1:
        return rec | {'error': 'ambiguous Moon', 'moon': None}
    raise ValueError("How did we get here?")


def get_moon_cutout(remaining: dict, moonpad: int) -> list[slice]:
    moon_y, moon_x = next(iter(remaining.values()))
    y0, y1, x0, x1 = moon_y.min(), moon_y.max(), moon_x.min(), moon_x.max()
    return [
        slice(max(y0 - moonpad, 0), y1 + moonpad),
        slice(max(x0 - moonpad, 0), x1 + moonpad)
    ]


def mooncircle(labels, moonslice, moonlabel) -> dict:
    """
    Given a dict containing a label array (keyed "labels") and a tuple of
    Y/X slices giving the bounds of a detected "Moon" (keyed "moonlabel"),
    compute the convex hull of the detected Moon, and approximate its
    illuminated fraction by comparing the area of that hull to the area of its
    minimum enclosing circle (which represents an implied full Moon).
    """
    moonblob = labels[*moonslice] == moonlabel
    smoothblob = ndi.binary_dilation(moonblob, np.ones((5, 5))).astype('u1')
    hull = gethull(smoothblob)
    center, radius = cv2.minEnclosingCircle(hull)
    return {
        'smoothblob': smoothblob,
        'center': center,
        'radius': radius,
        'ratio': cv2.contourArea(hull) / (np.pi * radius ** 2)
    }
    

# TODO: don't magically provide good parameters. make them mess with it.
def extract_moon(
    arr: np.ndarray,
    threshold: float = 0.7,
    erosion: int = 4,
    bordersize: int = 50,
    moonpad: int = 60,
    min_moon_extent: int = 180,
    extended: bool = False
) -> dict:
    """
    Attempt to identify/locate a Moon in a 2D array taken from a GOES LUN
    radiance image. Takes a number of parameters for different steps of the
    extraction pipeline:
    * threshold: cutoff value for making initial threshold array
    * erosion: side length of square footprint for erosion operator applied
      to threshold array
    * bordersize: image border width for top/bottom/left/right label rejection
    * moonpad: number of "padding" pixels to add to the Moon cutout
    * min_moon_extent: minimum extent, along both X and Y axes, for identifying
      a label as the Moon
    * extended: return extended information?

    TODO: internal documentation, split out, explain, blah blah
    """
    morph = (arr.filled(0) > threshold)
    if erosion is not None:
        morph = ndi.binary_erosion(morph, np.ones((erosion, erosion)))
    morph = morph.astype('u1')
    labels, n_labels = ndi.label(morph)
    remaining, label_rejects, label_indices = filter_labels(
        arr, labels, bordersize, min_moon_extent
    )
    rec = {'n_labels': n_labels, 'reject_reasons': label_rejects}
    if extended is True:
        rec |= {'labels': labels, 'label_indices': label_indices}
    if len(remaining) != 1:
        return moon_extraction_error(rec, len(remaining))
    moonlabel = next(iter(remaining.keys()))
    moonslice = get_moon_cutout(remaining, moonpad)
    rec |= mooncircle(labels, moonslice, moonlabel)
    if extended is True:
        rec['moonlabel'] = moonlabel
        rec['moonslice'] = moonslice
    return rec | {'error': None, 'moon': arr[*moonslice]}